<a href="https://colab.research.google.com/github/Jpedrogama/DataScience-Projeto-Imoveis/blob/main/lugar_certo/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re

In [3]:
url_apartamentos = []

base_url = "https://www.lugarcerto.com.br"
query_url = f"{base_url}/busca/compra-e-venda/go/goiania/apartamento"


In [4]:
href = ""
while(href != "?offset="):
  query = requests.get(f"{query_url}{href}").text
  soup = BeautifulSoup(query, 'html.parser')
  class_proprieries = soup.findAll("a", title="Ver detalhes deste anúncio")

  for link in class_proprieries:
    url_apartamentos.append(link['href'])
        
  href = soup.find("a", title="Próxima página")
  href = href['href']
  href = (re.sub("[^0-9]", "", href))
  print(f"Página:{int(href)/20}")
  href= f"?offset={href}"
print('over')


Página:1.0
Página:2.0
Página:3.0
Página:4.0
Página:5.0
Página:6.0
Página:7.0
Página:8.0
Página:9.0
Página:10.0
Página:11.0
Página:12.0
Página:13.0
Página:14.0
Página:15.0
Página:16.0
Página:17.0
Página:18.0
Página:19.0
Página:20.0
Página:21.0
Página:22.0
Página:23.0
Página:24.0
Página:25.0
Página:26.0
Página:27.0
Página:28.0
Página:29.0
Página:30.0
Página:31.0
Página:32.0
Página:33.0
Página:34.0
Página:35.0
Página:36.0
Página:37.0
Página:38.0
Página:39.0
Página:40.0
Página:41.0
Página:42.0
Página:43.0
Página:44.0
Página:45.0
Página:46.0
Página:47.0
Página:48.0
Página:49.0
Página:50.0
Página:51.0
Página:52.0
Página:53.0
Página:54.0
Página:55.0
Página:56.0
Página:57.0
Página:58.0
Página:59.0
Página:60.0
Página:61.0
Página:62.0
Página:63.0
Página:64.0
Página:65.0
Página:66.0
Página:67.0
Página:68.0
Página:69.0
Página:70.0
Página:71.0
Página:72.0
Página:73.0
Página:74.0
Página:75.0
Página:76.0
Página:77.0
Página:78.0
Página:79.0
Página:80.0
Página:81.0
Página:82.0
Página:83.0
Página:84.0
P

TypeError: ignored

In [5]:
print(len(url_apartamentos))

6776


In [6]:
from datetime import datetime

In [7]:
infos = []

In [8]:
def scrape_data(url):

  pagina_apartamento = requests.get(url).text
  soup = BeautifulSoup(pagina_apartamento, 'html.parser')

  descricao = []
  conteudo = []
  map_item = []

  m2 = None
  bedrooms = None
  suite = None
  parking = None
  bathrooms = None

  try:
    itens_descricao = soup.findAll("span", attrs={"class":"item-descricao"})
    itens = soup.findAll("span", attrs={"class":"item-descricao-conteudo"})

    for item in itens_descricao:
      descricao.append(item.text.strip())

    for item in itens:
      conteudo.append(item.text.strip())

    for item in range(len(descricao)):
      map_item.append((descricao[item], conteudo[item]))

    for i in range(len(map_item)):
      if (map_item[i][0] == 'ÁREA'):
        m2 = int(re.sub("[^0-9]", "", map_item[i][1]))
      if (map_item[i][0] == 'QUARTOS' or map_item[i][0] == 'QUARTO'):
        bedrooms = int(re.sub("[^0-9]", "", map_item[i][1]))
      if (map_item[i][0] == 'SUÍTES' or map_item[i][0] == 'SUÍTE'):
        suite = int(re.sub("[^0-9]", "", map_item[i][1]))
      if (map_item[i][0] == 'VAGAS' or map_item[i][0] == 'VAGA'):
        parking = int(re.sub("[^0-9]", "", map_item[i][1]))
      if (map_item[i][0] == 'BANHEIROS' or map_item[i][0] == 'BANHEIRO'):
        bathrooms = int(re.sub("[^0-9]", "", map_item[i][1]))
  except:
    m2 = None
    bedrooms = None
    suite = None
    parking = None
    bathrooms = None

  try:
    price = soup.find("li", attrs={"class":"itenscustos__valor"}).text
    price = float(re.sub("[^0-9]", "", price))/100
  except:
    price = None

  try:
    condominio = soup.find("li", attrs={"class": "itenscustos__taxas"}).find(text=True, recursive=False)
    condominio = float(re.sub("[^0-9]", "", condominio))/100
  except:
    condominio = None

  try:
    iptu = soup.find("li", attrs={"class": "itenscustos__taxas"}).find("span", attrs={"class": "clearfix-sm"}, text=True, recursive=False).text
    iptu = float(re.sub("[^0-9]", "", iptu))/100
  except:
    iptu = None

  try:
    location = soup.find("script", text=re.compile("window.detalheanuncio="))
    location = location.contents[0]
    location = location.split(sep='"lc_location":')
    location = location[1].split(sep='"')
    longitude, latitude = location[1].split(sep=',')
  except:
    longitude = None
    latitude = None

  try:
    address = soup.find("span", attrs={"class": "resultados-da-busca-localizacao"}).text
    final_address, state = address.split(sep="-")
    street, number, neighborhood, city = final_address.split(sep=",")
    address = street + number
    address = address.strip()
    state = state.strip()
    neighborhood = neighborhood.strip()
    city = city.strip()
  except:
    address = None
    state = None
    neighborhood = None
    city = None

  try:
    ad_anunciante = soup.find("div", attrs={"id": "js-contate-o-anunciante"}).find("a")['href']
    request_seller = requests.get(ad_anunciante).text
    seller_soup = BeautifulSoup(request_seller, 'html.parser')
    amount = seller_soup.find("span", attrs={"class": "results-seach-number"}).text
    amount = int(re.sub("[^0-9]", "", amount))
    if(amount<=1):
      seller = False
    else:
      seller = True
  except:
    seller = None

  infos.append([url, price, iptu, condominio, m2, bedrooms, bathrooms, suite, parking, seller, address, neighborhood, city, state, (latitude, longitude), latitude, longitude, datetime.now() ])

In [9]:
apartamentos_1 = url_apartamentos[:1000]
apartamentos_2 = url_apartamentos[1001:2000]
apartamentos_3 = url_apartamentos[2001:3000]
apartamentos_4 = url_apartamentos[3001:4000]
apartamentos_5 = url_apartamentos[4001:5000]
apartamentos_6 = url_apartamentos[5001:6000]
apartamentos_7 = url_apartamentos[6001:]

print(len(apartamentos_1), len(apartamentos_2), len(apartamentos_3), len(apartamentos_4), len(apartamentos_5), len(apartamentos_6), len(apartamentos_7))

1000 999 999 999 999 999 775


In [10]:
from tqdm.notebook import tqdm_notebook as tqdm
import time

with tqdm(total=len(apartamentos_1)) as pbar:
    for apartamento in apartamentos_1:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)
        # if(pbar.n%20==0):
        #     time.sleep(7)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
with tqdm(total=len(apartamentos_2)) as pbar:
    for apartamento in apartamentos_2:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/999 [00:00<?, ?it/s]

In [16]:
with tqdm(total=len(apartamentos_3)) as pbar:
    for apartamento in apartamentos_3:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/999 [00:00<?, ?it/s]

In [17]:
with tqdm(total=len(apartamentos_4)) as pbar:
    for apartamento in apartamentos_4:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/999 [00:00<?, ?it/s]

In [18]:
with tqdm(total=len(apartamentos_5)) as pbar:
    for apartamento in apartamentos_5:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/999 [00:00<?, ?it/s]

In [19]:
with tqdm(total=len(apartamentos_6)) as pbar:
    for apartamento in apartamentos_6:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/999 [00:00<?, ?it/s]

In [20]:
with tqdm(total=len(apartamentos_7)) as pbar:
    for apartamento in apartamentos_7:
        query_url = apartamento
        scrape_data('https:'+query_url)
        pbar.update(1)

  0%|          | 0/775 [00:00<?, ?it/s]

In [23]:
import pandas as pd

table = pd.DataFrame(infos, columns=['url','price' , 'iptu','condominio',  'm2', 'bedrooms', 'bathrooms', 'suite', 'parking', 'seller' , 'address', 'neighborhood', 'city', 'state', 'lat_lon', 'latitude', 'longitude', 'horario_extracao' ])
table

,url,price,iptu,condominio,m2,bedrooms,bathrooms,suite,parking,seller,address,neighborhood,city,state,lat_lon,latitude,longitude,horario_extracao
0,https://www.lugarcerto.com.br/imovel/apartamen...,444000.0,NaN,NaN,69.0,2.0,2.0,1.0,1.0,True,Av. T8 Esquina Com Rua T30 9,Setor Bueno,Goiânia,GO,"(-49.2780936588, -16.6977720213)",-49.2780936588,-16.6977720213,2021-11-01 12:28:21.518290
1,https://www.lugarcerto.com.br/imovel/apartamen...,890000.0,0.0,1650.0,241.0,4.0,3.0,2.0,4.0,True,Avenida 136 0,Setor Marista,Goiânia,GO,"(-49.2612362, -16.6985349)",-49.2612362,-16.6985349,2021-11-01 12:28:24.758190
2,https://www.lugarcerto.com.br/imovel/apartamen...,805000.0,NaN,500.0,125.0,3.0,4.0,3.0,2.0,True,None,None,None,None,"(-49.2770599, -16.7082503)",-49.2770599,-16.7082503,2021-11-01 12:28:27.329841
3,https://www.lugarcerto.com.br/imovel/apartamen...,4700000.0,3000.0,2000.0,404.0,4.0,6.0,4.0,5.0,True,34 4,Setor Marista,Goiânia,GO,"(-49.266272136, -16.6953891353)",-49.266272136,-16.6953891353,2021-11-01 12:28:30.056918
4,https://www.lugarcerto.com.br/imovel/apartamen...,1800000.0,0.0,0.0,254.0,4.0,6.0,4.0,3.0,True,Rua 1141 0,Setor Marista,Goiânia,GO,"(-49.276561, -16.629414)",-49.276561,-16.629414,2021-11-01 12:28:33.196279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6769,https://www.lugarcerto.com.br/imovel/apartamen...,840000.0,NaN,NaN,159.0,3.0,4.0,3.0,3.0,True,None,None,None,None,"(None, None)",None,None,2021-11-01 18:19:22.256674
6770,https://www.lugarcerto.com.br/imovel/apartamen...,NaN,NaN,NaN,110.0,3.0,3.0,3.0,2.0,True,None,None,None,None,"(None, None)",None,None,2021-11-01 18:19:25.038003
6771,https://www.lugarcerto.com.br/imovel/apartamen...,NaN,NaN,NaN,218.0,4.0,6.0,4.0,3.0,True,None,None,None,None,"(None, None)",None,None,2021-11-01 18:19:27.446751
6772,https://www.lugarcerto.com.br/imovel/apartamen...,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.0,True,None,None,None,None,"(None, None)",None,None,2021-11-01 18:19:29.846829


In [27]:
table.to_csv('Lugar Certo - Apartamentos')

AttributeError: ignored